In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pickle
import numpy as np
from tensorflow.keras.layers import Input
    

In [2]:
data = pd.read_csv("Churn_Modelling.csv")
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown= 'ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns = onehot_encoder_geo.get_feature_names_out())

data = pd.concat([data.drop('Geography', axis= 1), geo_encoded_df], axis= 1)

X = data.drop('Exited', axis = 1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2, random_state= 42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


## Save the encoder and scalers for later use
with open('label_encoder_gender.pkl', 'wb') as f:
    pickle.dump(label_encoder_gender, f)

with open('onehot_encoder_geo.pkl', 'wb') as f:
    pickle.dump(onehot_encoder_geo, f)
    
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
    
    

In [3]:
# Define a function to build the Keras model
def build_model(neurons=32, layers=1, input_dim=None):
    
    # Use the functional API
    inputs = Input(shape=(input_dim,))
    x = Dense(neurons, activation='relu')(inputs)
    
    for _ in range(layers - 1):
        x = Dense(neurons, activation='relu')(x)
    
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model
# Define the parameter grid
param_grid = {
    'neurons': [32, 64],  # Reduced for testing
    'layers': [1, 2], 
    'epochs': [50]
}

# Manual grid search with cross-validation
best_score = 0
best_params = {}
kf = KFold(n_splits=3, shuffle=True, random_state=42)

# Convert y_train to numpy if it's a pandas Series
if hasattr(y_train, 'values'):
    y_train_values = y_train.values
else:
    y_train_values = y_train

for neurons in param_grid['neurons']:
    for layers in param_grid['layers']:
        for epochs in param_grid['epochs']:
            cv_scores = []
            
            for train_idx, val_idx in kf.split(X_train):
                X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
                y_train_fold, y_val_fold = y_train_values[train_idx], y_train_values[val_idx]
                
                # Build and train model
                model = build_model(neurons=neurons, layers=layers, input_dim=X_train.shape[1])
                
                # Add early stopping to prevent overfitting
                early_stop = tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss', patience=5, restore_best_weights=True
                )
                
                model.fit(
                    X_train_fold, y_train_fold,
                    epochs=epochs,
                    batch_size=32,
                    validation_data=(X_val_fold, y_val_fold),
                    callbacks=[early_stop],
                    verbose=0
                )
                
                # Evaluate model
                y_pred = (model.predict(X_val_fold) > 0.5).astype(int)
                score = accuracy_score(y_val_fold, y_pred.flatten())
                cv_scores.append(score)
            
            avg_score = np.mean(cv_scores)
            print(f"neurons={neurons}, layers={layers}, epochs={epochs}: {avg_score:.4f}")
            
            if avg_score > best_score:
                best_score = avg_score
                best_params = {'neurons': neurons, 'layers': layers, 'epochs': epochs}

print(f"\nBest: {best_score:.4f} using {best_params}")

# Train final model with best parameters
final_model = build_model(
    neurons=best_params['neurons'], 
    layers=best_params['layers'],
    input_dim=X_train.shape[1]
)

# Add early stopping for final model training too
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)

final_model.fit(
    X_train, y_train_values,
    epochs=best_params['epochs'],
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

# Evaluate on test set
y_test_np = y_test.values if hasattr(y_test, 'values') else y_test
predictions = final_model.predict(X_test)
test_predictions = (predictions > 0.5).astype(int)
test_acc = accuracy_score(y_test_np, test_predictions)
print(f"Test accuracy: {test_acc:.4f}")

84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 988us/step
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 898us/step
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
neurons=32, layers=1, epochs=50: 0.8546
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 907us/step
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 841us/step
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
neurons=32, layers=2, epochs=50: 0.8572
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 959us/step
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
neurons=64, layers=1, epochs=50: 0.8550
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 961us/step
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
neurons=64, layers=2, epochs=50: 0.8549

Best: 0.8572 using {'neurons': 32, 'layers': 2, 'epochs': 50}
Epoch 1/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8016 - loss: 0.5128 - val_accuracy: 0.8263 - val_loss: 0.4117
Epoch 2/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8091 - loss: 0.4298 - val_accuracy: 0.8381 - val_loss: 0.3864
Epoch 3/50
200/200 ━━━━━━━━━━